# Summarize large text

Anton Antonov   
August 2025

----

## Setup

In [10]:
use LLM::Graph;
use Graph;
use Data::Importers;

In [11]:
my $engine = 'neato';

neato

----

## LLM graph

In [12]:
llm-prompt("TitleSuggest")

-> $a = "", $b = "journal article" { #`(Block|6004336970488) ... }

In [ ]:
sink my %rules =
TypeOfInput => sub ($_) {
        "Determine the input type of\n\n$_.\n\nThe result should be one of: 'Text', 'URL', 'FilePath', or 'Other'."  ~ 
        llm-prompt('NothingElse')('single string')
    },

IngestText =>  { eval-function => sub ($TypeOfInput, $_) { $TypeOfInput ~~ / URL | FilePath/ ?? data-import($_) !! $_} },

Title => { 
    eval-function => sub ($IngestText, $with-title = Whatever) { $with-title.defined ?? $with-title !! llm-synthesize(llm-prompt("TitleSuggest")($IngestText, 'article')) },
},

Summary => sub ($IngestText) { llm-prompt("Summarize")() ~ "\n\n$IngestText" },

TopicsTable => sub ($IngestText) { llm-prompt("ThemeTableJSON")($IngestText, 'article', 20) },

ThinkingHats => sub ($IngestText) { llm-prompt("ThinkingHatsFeedback")($IngestText, 'HTML') },

MindMap => sub ($IngestText) { 
    [
        "Create a concise mermaid-js mind-map for the text:",
        $IngestText,
        "All node messages should be in quotes.",
        llm-prompt("NothingElse")("correct mermaid-js")
    ]
},

Report => { eval-function => 
    sub ($Title, $Summary, $TopicsTable, $MindMap, $ThinkingHats) { 
        [
            "# $Title",
            '### *LLM summary report*',
            '## Summary',
            $Summary,
            '## Topics',
            to-html(
                from-json($TopicsTable.subst(/ ^ '```json' | '```' $/):g),
                field-names => <theme content>,
                align => 'left'),
            "## Mind map",
            $MindMap,
            '## Thinking hats',
            $ThinkingHats.subst(/ ^ '```html' | '```' $/):g
        ].join("\n\n")
    } 
},

ExportAndOpen => {
    eval-function => sub ($Report) {
       spurt('./Report.md', $Report);
       shell "open ./Report.md" 
    },
    test-function => -> $export-and-open { $export-and-open  ~~ Bool:D && $export-and-open  || $export-and-open .Str.lc ∈ <true yes open> }
}
;

Make the graph:

In [24]:
my $gCombinedSummary = LLM::Graph.new(%rules)

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

----

## Full computation

URL in text statistics:

In [21]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/Graph-neat-examples-in-Raku-Set-3-YouTube.txt';
my $txtFocus = data-import($url);

text-stats($txtFocus)

(chars => 5861 words => 1158 lines => 156)

Computation:

In [31]:
$gCombinedSummary.eval($url, 
with-title => '«Graph» neat examples, set 3', 
:export-and-open,
Summary => 'Summary',
TypeOfInput => 'Other',
IngestText => 'IngestText',
ThinkingHats => 'ThinkingHats',
TopicsTable => '["TopicsTable"]',
MindMap => 'MindMap',
)

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

Show the corresponding graph:

In [32]:
#% html
$gCombinedSummary.dot(node-width => 1.6 ):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- IngestText -->
 
 IngestText 
 
 IngestText 
 
<!-- $_->IngestText -->
 
 $_->IngestText 
 
 
 
<!-- TypeOfInput -->
 
 TypeOfInput 
 
 TypeOfInput 
 
<!-- $_->TypeOfInput -->
 
 $_->TypeOfInput 
 
 
 
<!-- ExportAndOpen -->
 
 ExportAndOpen 
 
 ExportAndOpen 
 
<!-- MindMap -->
 
 MindMap 
 
 MindMap 
 
<!-- IngestText->MindMap -->
 
 IngestText->MindMap 
 
 
 
<!-- Summary -->
 
 Summary 
 
 Summary 
 
<!-- IngestText->Summary -->
 
 IngestText->Summary 
 
 
 
<!-- ThinkingHats -->
 
 ThinkingHats 
 
 ThinkingHats 
 
<!-- IngestText->ThinkingHats -->
 
 IngestText->ThinkingHats 
 
 
 
<!-- Title -->
 
 Title 
 
 Title 
 
<!-- IngestText->Title -->
 
 IngestText->Title 
 
 
 
<!-- TopicsTable -->
 
 TopicsTable 
 
 TopicsTable 
 
<!-- IngestText->TopicsTable -->
 
 IngestText->TopicsTable 
 
 
 
<!-- Report -->
 
 Report 
 
 Report 
 
<!-- MindMap->Report -->
 
 MindMap->Report 
 
 
 
<!-- Report->ExportAndOpen -->
 
 Report->ExportAndOpen 
 
 
 
<!-- Summary->Report -->
 
 Summary->Report 
 
 
 
<!-- ThinkingHats->Report -->
 
 ThinkingHats->Report 
 
 
 
<!-- Title->Report -->
 
 Title->Report 
 
 
 
<!-- TopicsTable->Report -->
 
 TopicsTable->Report 
 
 
 
<!-- TypeOfInput->IngestText -->
 
 TypeOfInput->IngestText 
 
 
 
<!-- export-and-open -->
 
 export-and-open 
 
 export-and-open 
 
<!-- export-and-open->ExportAndOpen -->
 
 export-and-open->ExportAndOpen 
 
 
 
<!-- with-title -->
 
 with-title 
 
 with-title 
 
<!-- with-title->Title -->
 
 with-title->Title

Final result:

In [29]:
#% markdown
$gCombinedSummary.rules<Report><result>.subst(/'```html' | '```' $/):g

No such method 'subst' for invocant of type 'Any'

---

## Partial evaluation

Drop the results in `LLM::Graph` computed above:

In [10]:
$gCombinedSummary.drop-results

LLM::Graph(size => 7, nodes => IngestText, MindMap, Report, Summary, ThinkingHats, TopicsTable, TypeOfInput)

Here the are normalized nodes without results:

In [11]:
.say for |$gCombinedSummary.rules

Summary => {eval-function => sub { }, input => [IngestText], test-function-input => [], wrapper => Routine::WrapHandle.new}
ThinkingHats => {eval-function => sub { }, input => [IngestText], test-function-input => [], wrapper => Routine::WrapHandle.new}
IngestText => {eval-function => sub { }, input => [TypeOfInput], test-function-input => []}
TypeOfInput => {eval-function => sub { }, input => [], test-function-input => [], wrapper => Routine::WrapHandle.new}
Report => {eval-function => sub { }, input => [ThinkingHats MindMap Summary TopicsTable], test-function-input => []}
MindMap => {eval-function => sub { }, input => [IngestText], test-function-input => [], wrapper => Routine::WrapHandle.new}
TopicsTable => {eval-function => sub { }, input => [IngestText], test-function-input => [], wrapper => Routine::WrapHandle.new}


----

## Special DOT representation

In [9]:
sub dot-svg($input, Str:D :$engine = 'dot', Str:D :$format = 'svg') {
        my $temp-file = $*TMPDIR.child("temp-graph.dot");
        $temp-file.spurt: $input;
        my $svg-output = run($engine, $temp-file, "-T$format", :out).out.slurp-rest;
        unlink $temp-file;
        return $svg-output;
    }

&dot-svg

In [91]:
$gCombinedSummary.create-graph

Graph(vertexes => 8, edges => 11, directed => True)

In [18]:
#% html
$gCombinedSummary.graph.edges(:dataset)
==> to-html()

to from weight Report Summary 1 IngestText TypeOfInput 1 TypeOfInput $_ 1 IngestText $_ 1 ThinkingHats IngestText 1 TopicsTable IngestText 1 Summary IngestText 1 MindMap IngestText 1 Report MindMap 1 Report ThinkingHats 1 Report TopicsTable 1

In [21]:
$gCombinedSummary.rules.grep({ not $_.value<wrapper>:exists })».key

(IngestText Report)

In [22]:
#% html
$gCombinedSummary.graph.dot(:weights, engine => 'dot', graph-size => 6, :8node-font-size, node-shape => 'ellipse', node-width => 0.8 )
==> { 
    my @proc = $gCombinedSummary.rules.grep({ not $_.value<wrapper>:exists })».key;
    my $res = $_;

    for @proc -> $p {
        $res .= subst('"' ~ $p ~ '"; ', '"' ~ $p ~ '" [shape=box];')
    }

    my @inp = ($gCombinedSummary.graph.vertex-list (-) $gCombinedSummary.rules.keys).keys;

    for @inp -> $p {
        $res .= subst('"' ~ $p ~ '"; ', '"' ~ $p ~ '" [shape=parallelogram];')
    }

    $res
    .subst('[weight=2, label=2]', '[weight=2, style=dashed]', :g)
    .subst('[weight=1, label=1]', '[weight=1]', :g)
    .subst('[weight=3, label=3]', '[weight=1]', :g)
    }()
==> { dot-svg($_) }()

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- $_ -->
 
 $_ 
 
 $_ 
 
<!-- IngestText -->
 
 IngestText 
 
 IngestText 
 
<!-- $_->IngestText -->
 
 $_->IngestText 
 
 
 
<!-- TypeOfInput -->
 
 TypeOfInput 
 
 TypeOfInput 
 
<!-- $_->TypeOfInput -->
 
 $_->TypeOfInput 
 
 
 
<!-- MindMap -->
 
 MindMap 
 
 MindMap 
 
<!-- IngestText->MindMap -->
 
 IngestText->MindMap 
 
 
 
<!-- Summary -->
 
 Summary 
 
 Summary 
 
<!-- IngestText->Summary -->
 
 IngestText->Summary 
 
 
 
<!-- ThinkingHats -->
 
 ThinkingHats 
 
 ThinkingHats 
 
<!-- IngestText->ThinkingHats -->
 
 IngestText->ThinkingHats 
 
 
 
<!-- TopicsTable -->
 
 TopicsTable 
 
 TopicsTable 
 
<!-- IngestText->TopicsTable -->
 
 IngestText->TopicsTable 
 
 
 
<!-- Report -->
 
 Report 
 
 Report 
 
<!-- MindMap->Report -->
 
 MindMap->Report 
 
 
 
<!-- Summary->Report -->
 
 Summary->Report 
 
 
 
<!-- ThinkingHats->Report -->
 
 ThinkingHats->Report 
 
 
 
<!-- TopicsTable->Report -->
 
 TopicsTable->Report 
 
 
 
<!-- TypeOfInput->IngestText -->
 
 TypeOfInput->IngestText